In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import gzip, pickle, pickletools
import plotly.express as px
import cufflinks as cf
import chart_studio.plotly as py
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
#to make everything locally
cf.go_offline()

In [15]:
data = pd.read_csv("D:\data science\ineuron\Project\Business Analytics\Data_given\gpsfinal.csv")
data.head()

,Rating,Reviews,Size_New,Installs_New,Price_New,Category_new,Content_Rating_new
0,4.1,97,19.0,10000,0,0,1
1,3.9,57,14.0,500000,0,0,1
2,4.7,42,8.7,5000000,0,0,1
3,4.5,92,25.0,50000000,0,0,4
4,4.3,57,2.8,100000,0,0,1


In [16]:
px.box(data, y = 'Size_New')

In [17]:
data = data[data['Size_New']<=67]

In [18]:
px.box(data, y = 'Installs_New')

In [19]:
data = data[data['Installs_New']<=1000000]

In [22]:
### Model Training#####
X = data.drop(columns = ['Rating'])
Y = data['Rating']>4.2
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=.15 , random_state=45)



# Scale the data to be between -1 and 1
scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)


param_grid = {'n_estimators':[100,200,400,500,800,1000,1500],
              'min_samples_leaf':[1,2,3],
              'max_features' : ["auto", "sqrt", "log2"],
              'criterion':['gini','entropy']
             }
rfr = RandomForestClassifier(oob_score =True, n_jobs=5)
rfr_grid = GridSearchCV(rfr, cv = 5, param_grid=param_grid)
rfr_grid.fit(train_X,train_y)
rfr_grid.best_estimator_

RandomForestClassifier(max_features='log2', min_samples_leaf=3,
                       n_estimators=200, n_jobs=5, oob_score=True)

In [24]:
rfr_grid.score(train_X,train_y)

0.9116479152878888

In [ ]:
rfr_grid.score(train_X,train_y)

In [9]:
filepath = "random_forest.pkl"
with gzip.open(filepath, "wb") as f:
    pickled = pickle.dumps(rfr_grid)
    optimized_pickle = pickletools.optimize(pickled)
    f.write(optimized_pickle)

In [10]:
# Loop is used to get more generalized result
total_sum = []
for i in range(10):
    # Hypertuning of parameters for better prediction
    forest_model = RandomForestRegressor(n_estimators=100, max_features=3, min_samples_leaf=10)
    forest_model.fit(train_X,train_y)
    
    y_pred = forest_model.predict(test_X)
    print("r2_score value ", r2_score(test_y,y_pred))
    print("rmse value ", np.sqrt(mean_squared_error(test_y,y_pred)))
    print(mean_absolute_error(test_y,y_pred))
    forest_model.score(test_X,test_y)
    total_sum.append(np.sqrt(mean_squared_error(test_y,y_pred)))
    

r2_score value  0.08094470423200639
rmse value  0.5133738123771443
0.3530251012795524
r2_score value  0.08795256348825475
rmse value  0.5114128118824794
0.35250613686081234
r2_score value  0.09063676212359062
rmse value  0.5106597013475141
0.351309605481089
r2_score value  0.08890303205016414
rmse value  0.5111462640659585
0.3523103305797619
r2_score value  0.08803194534493308
rmse value  0.5113905554842334
0.35149266921071154
r2_score value  0.08839164744992567
rmse value  0.511289693178628
0.35194940827130355
r2_score value  0.08868405506623689
rmse value  0.5112076859269706
0.3516749246607123
r2_score value  0.08836261073407603
rmse value  0.511297835960162
0.35191971702501906
r2_score value  0.09157326020659817
rmse value  0.5103966848759921
0.35117056059375307
r2_score value  0.08437705949741503
rmse value  0.5124142783930143
0.3515454434148044


In [20]:
Y = data['Rating']>4.2

In [21]:
Y

0       False
1       False
4        True
5        True
6       False
        ...  
9139    False
9140    False
9141     True
9142     True
9143    False
Name: Rating, Length: 7111, dtype: bool